In [ ]:
import numpy as np
from importlib import reload
import lib.wave_utils
from lib.wave_utils import *
module_plot = reload(lib.wave_utils)

---
Wave Equation (1d), partial derivative form:<br><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>$∂²u/∂t² = c² ∂²u/∂x²$</b><br>
<li>u(t,x) is the displacement of the wave at time t and position x</li>
<li>u[t_n, x_i] is the discretized version, where with input time t_n and position x_i</li>
<li>c is the wave speed (as determined by the medium).</li><br>

---
Finite Differences / Discretized Numeric Approach (e.g. using finite dt, dx)<br>
&nbsp;&nbsp;&nbsp;Use Central Difference Approximation to approx second derivative. Derived from taylor series approx (using first three terms) - do expansion at u(t) and eval at u(t-dt) and u(t+dt), then solve for the second derivatives (the first derivatives cancel out)...<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>$∂²u/∂t² ≈ (u(t + Δt, x) - 2u(t, x) + u(t - Δt, x)) / Δt²$</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>$∂²u/∂x² ≈ (u(t, x + Δx) - 2u(t, x) + u(t, x - Δx)) / Δx²$</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</b>discretized versions for our sim at point tn, where displacement is u[t, x]...</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>$∂²u/∂t² ≈ (u[t_n + 1, x_i] - 2u[t_n, x_i] + u[t_n - 1, x_i]) / Δt²$</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>$∂²u/∂x² ≈ (u[t_n, x_i + 1] - 2u[t_n, x_i] + u[t_n, x_i - 1]) / Δx²$</b><br>

---
Substitute into the Wave Equation:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>$(u[t_n + 1, x_i] - 2u[t_n, x_i] + u[t_n - 1, x_i]) / Δt² = c² (u[t_n, x_i + 1] - 2u[t_n, x_i] + u[t_n, x_i - 1]) / Δx²$</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...and solve for the next offset:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>$u[t_n + 1, x_i] = 2(1 - r²)u[t_n, x_i] - u[t_n - 1, x_i] + r²(u[t_n, x_i + 1] + u[t_n, x_i - 1])$</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...where <b>$r = c Δt/Δx$</b> (called the Courant number, note that c is the speed of the wave and Δt & Δx are properties of the simulation)<br>

---

In [ ]:
# physical parameters
L = 0.65    # length of the string (m)
T = 0.04    # total simulation time (in seconds)
c = 340.0   # wave speed (m/s) - for guitar string [100, 400] m/s is reasonable (depends on string density & tension)

# simulation parameters calculate dx and dt based on CFL (Courant-Friedrichs-Lewy) condition
nx = 150    # number of points in x (along the length of the string)
dx = L / (nx - 1)   # calculate dx and dt based on CFL (Courant-Friedrichs-Lewy) condition
dt = 0.9 * dx / c   # need to keep it stable, so tie dt to dx (r needs to be less than 1)
r = c * dt / dx
nt = int(T / dt) + 1

# initialize solution arrays
u = np.zeros((nt, nx), dtype=float)  # wave displacement, scalar function u(t,x) (so, vert displacement at each time/position)
v = np.zeros((nt, nx), dtype=float)  # wave displacement's velocity, ∂u/∂t (at each time/position pair)

# initial conditions, set x values across the length of the string, then initial displacement & velocity
x = np.linspace(0, L, nx)
maxInitVel = 100
xPeakPos = nx//2
v[0, :] = np.concatenate([np.linspace(0, maxInitVel, xPeakPos), np.linspace(maxInitVel, 0, nx - xPeakPos)])  # initial velocity (du/dt)
u[0, :] = v[0, :] * (dt * 0)        # initial displacement (maybe start with some initial offset...or not)

# figure out u at time 1 using the initial velocities (we don't need v after this, it is then encoded in the previous displacements)
u[1, 1:-1] = u[0, 1:-1] + dt * v[0, 1:-1] + (dt**2 / 2) * (c**2) * (u[0, 2:] - 2*u[0, 1:-1] + u[0, :-2]) / dx**2

# boundary conditions (first and last x values, across all time points)
u[:, 0] = np.zeros(nt)
u[:, -1] = np.zeros(nt)

# simulate
for t_n in range(1, nt - 1):
    for x_i in range(1, nx - 1):
        u[t_n + 1, x_i] = 2 * (1 - r**2) * u[t_n, x_i] - u[t_n - 1, x_i] + r**2 * (u[t_n, x_i + 1] + u[t_n, x_i - 1])
        v[t_n + 1, x_i] = (u[t_n+1, x_i] - u[t_n, x_i]) / dt # approximate velocity (not really needed, but still calculating for fun)

    # apply boundary conditions at each time step
    u[t_n + 1, 0] = 0.0
    u[t_n + 1, -1] = 0.0

# show animated plot
anim = showAnim("Displacement Along String's Length", x, u)
HTML(anim.to_jshtml())

In [ ]:
# find frequency (plot displacement vs time @ the midpoint of x, you can use showGraph(...) to graph t vs u)
# TODO

# calculate it based on peaks (you can use calcSingalFrequency(...) to help with this)
# TODO

In [ ]:
# play the note at that frequence (you can use playNote(...) to help with this)
# TODO

---
Ok, let's add damping - the partial derivative is...<br><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>$∂²u/∂t² + γ ∂u/∂t = c² ∂²u/∂x²$</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$u(t,x)$ is the displacement of the wave at time t and position $x$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$u[t_n, x_i]$ is the discretized version, where with input time t_n and position $x_i$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$c$ is the wave speed (as determined by the medium).<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$γ$ (gamma) is the damping term, proportional to the first time derivative (velocity), for our time scale, start with 50 (very heavy damping).<br>

---
Using the same technique and solving for $u[t_n + 1, x_i]$:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>$u[t_n + 1, x_i] = (2 - γ * dt) u[t_n, x_i] - (1 - γ * dt) u[t_n - 1, x_i] +
r² (u[t_n, x_i + 1] - 2 u[t_n, x_i] + u[t_n, x_i - 1])$</b><br>

In [ ]:
# create an animation with damping
damping = 50

# simulate
# TODO

# show animated plot
# TODO